# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos



El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

Integrantes: Vickiann Jimenez, Gabriela Salazar, Jostin Vega

Descripción del Ejercicio



1. Procesamiento del Corpus:

    * Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).





In [1]:
# Importar las librerías necesarias

import xml.etree.ElementTree as ET
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re



# Definir las consultas

queries = {

    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",

    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",

    3: "Estrategias universitarias para reducir el estrés en estudiantes"

}



# Función para procesar el texto y extraer palabras clave

def process_text(text):

    # Convertir a minúsculas

    text = text.lower()

    # Reemplazar caracteres no alfanuméricos por espacios

    import re

    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)

    # Tokenizar y eliminar palabras vacías si es necesario

    tokens = text.strip().split()

    return set(tokens)



# Paso 1: Leer y parsear el archivo XML

def parse_corpus(xml_file):

    tree = ET.parse(xml_file)

    root = tree.getroot()

    corpus = {}

    for doc in root.findall('document'):

        doc_id = int(doc.get('id'))

        title = doc.find('title').text

        keywords = doc.find('keywords').text

        author = doc.find('author').text

        date = doc.find('date').text

        # Procesar las palabras clave

        keyword_set = process_text(keywords)

        corpus[doc_id] = {

            'title': title,

            'keywords': keyword_set,

            'author': author,

            'date': date

        }

    return corpus

In [ ]:
corpus = parse_corpus('03ranking_corpus.xml')

corpus

{1: {'title': 'El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.',
  'keywords': {'crónica',
   'médica',
   'salud',
   'tecnología',
   'telemedicina',
   'tratamiento'},
  'author': 'Dr. Juan Pérez',
  'date': '2023-01-15'},
 2: {'title': 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.',
  'keywords': {'académico',
   'estudiantes',
   'mental',
   'nutrición',
   'rendimiento',
   'salud'},
  'author': 'Dra. María López',
  'date': '2023-02-10'},
 3: {'title': 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.',
  'keywords': {'amistad',
   'bienestar',
   'campus',
   'estudiantil',
   'relaciones',
   'sociales'},
  'author': 'Miguel Rodríguez',
  'date': '2023-03-05'},
 4: {'title': 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.',
  'keywords': {'academia',
   'bibliotecas',
   'investigación',
   'recursos

2. Procesamiento de las Consultas:

    * Definir las consultas proporcionadas.

    * Extraer las palabras clave de cada consulta.




In [4]:
# Paso 2: Procesamiento de las Consultas
def process_queries(queries):
    processed_queries = {}
    for q_id, query in queries.items():
        processed_queries[q_id] = process_text(query)
    return processed_queries

# Procesar las consultas 
processed_queries = process_queries(queries)
print("\nConsultas:")
for q_id, keywords in processed_queries.items():
    print(f"Consulta {q_id}: {keywords}")


Consultas:
Consulta 1: {'universitarios', 'estudiantes', 'la', 'rendimiento', 'el', 'de', 'en', 'los', 'académico', 'impacto', 'salud', 'mental'}
Consulta 2: {'extracurriculares', 'el', 'emocional', 'en', 'actividades', 'campus', 'universitario', 'y', 'bienestar'}
Consulta 3: {'estrés', 'estudiantes', 'reducir', 'el', 'en', 'estrategias', 'universitarias', 'para'}



3. Cálculo de Relevancia:

    * Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.

    * Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

In [5]:
# Función de Similitud de Jaccard
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if union else 0

# Función de Similitud Coseno
def cosine_similarity(set1, set2):
    all_terms = list(set1.union(set2))
    vec1 = np.array([1 if term in set1 else 0 for term in all_terms])
    vec2 = np.array([1 if term in set2 else 0 for term in all_terms])
    dot_product = np.dot(vec1, vec2)
    magnitude = np.sqrt(np.dot(vec1, vec1)) * np.sqrt(np.dot(vec2, vec2))
    return dot_product / magnitude if magnitude else 0

# Paso 3: Calcular la relevancia entre cada consulta y cada documento
def calculate_relevance(corpus, queries):
    relevance_scores = {}
    for q_id, query_keywords in queries.items():
        relevance_scores[q_id] = {}
        for doc_id, doc_data in corpus.items():
            jaccard_score = jaccard_similarity(query_keywords, doc_data['keywords'])
            cosine_score = cosine_similarity(query_keywords, doc_data['keywords'])
            relevance_scores[q_id][doc_id] = {
                'jaccard': jaccard_score,
                'cosine': cosine_score
            }
    return relevance_scores

# Calcular las puntuaciones de relevancia
relevance_scores = calculate_relevance(corpus, processed_queries)
print("\nPuntuaciones de Relevancia (Similitud Jaccard y Coseno):")
for q_id, scores in relevance_scores.items():
    print(f"\nConsulta {q_id}:")
    for doc_id, score in scores.items():
        print(f"  Documento {doc_id}: Jaccard = {score['jaccard']:.4f}, Coseno = {score['cosine']:.4f}")



Puntuaciones de Relevancia (Similitud Jaccard y Coseno):

Consulta 1:
  Documento 1: Jaccard = 0.0588, Coseno = 0.1179
  Documento 2: Jaccard = 0.3846, Coseno = 0.5893
  Documento 3: Jaccard = 0.0000, Coseno = 0.0000
  Documento 4: Jaccard = 0.0000, Coseno = 0.0000
  Documento 5: Jaccard = 0.0000, Coseno = 0.0000
  Documento 6: Jaccard = 0.0588, Coseno = 0.1179
  Documento 7: Jaccard = 0.3846, Coseno = 0.5893
  Documento 8: Jaccard = 0.0588, Coseno = 0.1179
  Documento 9: Jaccard = 0.0556, Coseno = 0.1091
  Documento 10: Jaccard = 0.0000, Coseno = 0.0000
  Documento 11: Jaccard = 0.2857, Coseno = 0.4714
  Documento 12: Jaccard = 0.1333, Coseno = 0.2582
  Documento 13: Jaccard = 0.3125, Coseno = 0.4811
  Documento 14: Jaccard = 0.3571, Coseno = 0.5455
  Documento 15: Jaccard = 0.0588, Coseno = 0.1179
  Documento 16: Jaccard = 0.0556, Coseno = 0.1091
  Documento 17: Jaccard = 0.0588, Coseno = 0.1179
  Documento 18: Jaccard = 0.1875, Coseno = 0.3273
  Documento 19: Jaccard = 0.2500, Cose


4. Ranking de Documentos:

    * Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.

    * Mostrar el ranking de documentos para cada consulta.

In [6]:
# Paso 4: Crear el ranking basado en la relevancia
def rank_documents(relevance_scores, metric='cosine'):
    ranked_results = {}
    for q_id, scores in relevance_scores.items():
        ranked_results[q_id] = sorted(scores.items(), key=lambda item: item[1][metric], reverse=True)
    return ranked_results

# Generar y mostrar el ranking de documentos para cada consulta
ranked_results_jaccard = rank_documents(relevance_scores, metric='jaccard')
ranked_results_cosine = rank_documents(relevance_scores, metric='cosine')

for q_id in queries:
    print(f"\nConsulta {q_id}: {queries[q_id]}")
    print("Ranking (Similitud Jaccard):")
    for doc_id, score in ranked_results_jaccard[q_id]:
        print(f"  Documento {doc_id}: Puntuación de Jaccard = {score['jaccard']:.4f}")
    
    print("\nRanking (Similitud Coseno):")
    for doc_id, score in ranked_results_cosine[q_id]:
        print(f"  Documento {doc_id}: Puntuación de Coseno = {score['cosine']:.4f}")


Consulta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
Ranking (Similitud Jaccard):
  Documento 2: Puntuación de Jaccard = 0.3846
  Documento 7: Puntuación de Jaccard = 0.3846
  Documento 14: Puntuación de Jaccard = 0.3571
  Documento 13: Puntuación de Jaccard = 0.3125
  Documento 11: Puntuación de Jaccard = 0.2857
  Documento 19: Puntuación de Jaccard = 0.2500
  Documento 29: Puntuación de Jaccard = 0.2500
  Documento 18: Puntuación de Jaccard = 0.1875
  Documento 12: Puntuación de Jaccard = 0.1333
  Documento 21: Puntuación de Jaccard = 0.1176
  Documento 23: Puntuación de Jaccard = 0.1176
  Documento 26: Puntuación de Jaccard = 0.1176
  Documento 1: Puntuación de Jaccard = 0.0588
  Documento 6: Puntuación de Jaccard = 0.0588
  Documento 8: Puntuación de Jaccard = 0.0588
  Documento 15: Puntuación de Jaccard = 0.0588
  Documento 17: Puntuación de Jaccard = 0.0588
  Documento 24: Puntuación de Jaccard = 0.0588
  Documento 27: Puntuación d